In [5]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')
openai = OpenAI()

In [11]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Evaluate a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Math expression like '12 * 5 + 3'"
                    }
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_population",
            "description": "Get population information for a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "Population Count"
                    }
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for general information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Search query"
                    }
                },
                "required": ["query"]
            }
        }
    }
]


In [28]:
def get_population(city: str) -> int:
    """
    Mock population lookup
    """
    population_db = {
        "mumbai": 12442373,
        "pune": 3124458,
        "delhi": 16787941,
        "bangalore": 8443675
    }

    return population_db.get(city.lower(), 0)

def get_weather(city: str) -> str:
    weather_db = {
        "mumbai": "Sunny, 32°C",
        "pune": "Cloudy, 28°C",
        "bangalore": "Rainy, 24°C",
        "delhi": "Hazy, 35°C"
    }
    return weather_db.get(city.lower(), "Weather data not available")

def calculator(expression: str) -> str:
    try:
        result = eval(expression, {"__builtins__": {}}, math.__dict__)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"


In [34]:
import json

def calling_agent_with_tools(user_input: str):

    messages = [
        {"role": "system", "content": "You are a helpful assistant that uses tools when needed."},
        {"role": "user", "content": user_input}
    ]

    response = call_llm(messages)
    message = response.choices[0].message

    print("Model message:", message)

    # ✅ Append assistant message ONCE
    messages.append(message)

    if message.tool_calls:
        for tool_call in message.tool_calls:
            fn_name = tool_call.function.name
            fn_args = json.loads(tool_call.function.arguments)

            print(f"Calling tool: {fn_name} with args: {fn_args}")

            if fn_name == 'get_population':
                result = get_population(fn_args["city"])

            elif fn_name == 'calculator':
                result = calculator(fn_args["expression"])

            else:
                result = "Unknown tool"

            # ✅ Respond to EACH tool_call_id
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": fn_name,
                "content": str(result)
            })

        # ✅ Call model ONLY AFTER all tools are handled
        final_response = openai.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages
        )

        return final_response.choices[0].message.content

    return message.content

calling_agent_with_tools("What is the population of Mumbai? Also, what is 25 multiplied by 4 plus 10?")

Model message: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_nD5HhmW5Z6qDR0ovPAZ3KQCh', function=Function(arguments='{"city": "Mumbai"}', name='get_population'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_bvJWkeuVOuuDe6EpxKIOY0Cj', function=Function(arguments='{"expression": "25 * 4 + 10"}', name='calculator'), type='function')])
Calling tool: get_population with args: {'city': 'Mumbai'}
Calling tool: calculator with args: {'expression': '25 * 4 + 10'}


'The population of Mumbai is approximately 12,442,373. \n\nFor the calculation, 25 multiplied by 4 plus 10 equals 25 * 4 + 10 = 100 + 10 = 110.'